In [2]:
import pandas as pd
import json
import pickle
import re
from tqdm.auto import tqdm

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Читаем файл и достаём данные

In [241]:
skipped = 0

with open("lang-8-20111007-L1-v2.dat") as fp:
    records = []
    for row, item in tqdm(enumerate(fp.readlines())):
        try:
            item = json.loads(item)
        except Exception as e:
            skipped += 1
            continue
        lang = item[2]
        sentences = item[4]
        corrections = item[5]
        for sentence, correction in zip(sentences, corrections):
            for corr in correction:
                records.append(
                    {
                        'sentence': sentence,
                        'correction': corr,
                        'lang': lang,
                    }
                )

    df = pd.DataFrame.from_records(records)
    print("Skipped with errors: ", skipped)
        


579872it [00:44, 13079.43it/s]


Skipped with errors:  382


## Удаление ненужных тегов и мусора

In [243]:
# red - correction
# blue - insertion
# sline - removal
# bold - bold
def process(value):
    pattern = r'(\[sline\](.+?)\[\/sline])|(\[\/?f-red\])|(\[\/?f-blue\])|(\[\/?f-bold\])|(^[^а-яА-Я]*$)'
    mod_string = re.sub(pattern, '', value)
    pattern = r'(\d+[.)])|(\((.+?)\))|(\[(.+?)\])'
    mod_string = re.sub(pattern, '', mod_string)
    return mod_string.strip()

In [300]:
russian_df = df[df['lang'] == 'Russian']
print(len(russian_df))
russian_df = russian_df.dropna().reset_index(drop=True)
russian_df['processed_correction'] = russian_df['correction'].map(process)
russian_df['sentence'] = russian_df['sentence'].map(process)
patternDel = r"^[^а-яА-Я]*$"
filter = russian_df['processed_correction'].str.contains(patternDel)
russian_df = russian_df[~filter]
filter = russian_df['sentence'].str.contains(patternDel)
russian_df = russian_df[~filter]
pattern_latin = r"([a-zA-Z])"
filter = russian_df['processed_correction'].str.contains(pattern_latin)
russian_df = russian_df[~filter]
pattern_brackets = r"[\(\)\[\]\\\/]"
filter = russian_df['processed_correction'].str.contains(pattern_brackets)
russian_df = russian_df[~filter]

russian_df = russian_df[russian_df['sentence'] != russian_df['processed_correction']]
nan_value = float("NaN")
russian_df.replace("", nan_value, inplace=True)
russian_df.dropna(subset = ["processed_correction"], inplace=True)
russian_df.reset_index(inplace=True, drop=True)
print(len(russian_df))
russian_df = russian_df.sample(frac=1).reset_index(drop=True)
russian_df.head()

63657


/var/folders/7c/871yz62n4qj1dv2ks686jn600000gn/T/ipykernel_10221/2496945782.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filter = russian_df['processed_correction'].str.contains(pattern_latin)


53478


,sentence,correction,lang,processed_correction
0,"Поскольку я уже сказал, на будущей неделе я по...",[f-blue]Как[/f-blue] я уже [f-blue]говорил[/f-...,Russian,"Как я уже говорил , на будущей неделе я поеду..."
1,- я дам вам массаж,- я [f-red]сделаю[/f-red] вам массаж,Russian,- я сделаю вам массаж
2,"Поэтому, я волновалась до того, как моя голова...","Поэтому, я волновалась до того, [f-red]что[/f-...",Russian,"Поэтому, я волновалась до того, что моя голова..."
3,"В результате, у меня есть круглый, крупный живот.",[sline][f-blue][/f-blue][/sline][f-blue]Теперь...,Russian,Теперь у меня большой круглый живот.
4,Я не понимал все...,Я [f-bold]всё [/f-bold]не понимал [sline]все[/...,Russian,Я всё не понимал ...


In [301]:
pattern_not_russian = r"[\\\/]"
filter = russian_df['processed_correction'].str.contains(pattern_not_russian)
print(len(russian_df[filter]))
print(len(russian_df[filter]) / len(russian_df))
russian_df = russian_df[~filter]


0
0.0


In [310]:
# russian_df = russian_df.reset_index()
# del russian_df['index']
# del russian_df['lang']
russian_df.sample(5)

,sentence,correction,lang,processed_correction
2876,Виза должна прийти у меня завтра В ответ на о...,Виза должна прийти [f-blue]мне[/f-blue] завтра...,Russian,Виза должна прийти мне завтра В ответ на один...
40634,"Думаю, что важно поговорить о всём, включая пр...","Важно говорить о всём, включая проблемы. Но не...",Russian,"Важно говорить о всём, включая проблемы. Но не..."
19638,"У меня ""АР"" Японцкий оцмотр ча четверт.","У меня АР, [f-blue]экзамен по японскому[/f-blu...",Russian,"У меня АР, экзамен по японскому в четверг."
43548,Вы:..,Вы:.. А у меня гуманитарное образование. Сейча...,Russian,Вы:.. А у меня гуманитарное образование. Сейча...
35495,"Недавно я обнаружил, что в Кобе есть русский к...","Недавно я обнаружил, что в Кобе есть русский [...",Russian,"Недавно я обнаружил, что в Кобе есть русский б..."


In [311]:
index = 1824
print(russian_df['sentence'].iloc[index])
print(russian_df['correction'].iloc[index])
print(russian_df['processed_correction'].iloc[index])

СИЛНО устал!
СИЛ[f-blue]Ь[/f-blue]НО устал!
СИЛЬНО устал!


In [312]:
russian_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53478 entries, 0 to 53477
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   sentence              53478 non-null  object
 1   correction            53478 non-null  object
 2   lang                  53478 non-null  object
 3   processed_correction  53478 non-null  object
dtypes: object(4)
memory usage: 2.0+ MB


In [313]:
russian_df.to_csv('russian_gec_dataset_lang8.csv', sep=';', encoding="UTF-8", index=False)

In [3]:
data = pd.read_csv('russian_gec_dataset_lang8.csv', sep=';', encoding="UTF-8")
data.head(5)

,sentence,correction,lang,processed_correction
0,"Поскольку я уже сказал, на будущей неделе я по...",[f-blue]Как[/f-blue] я уже [f-blue]говорил[/f-...,Russian,"Как я уже говорил , на будущей неделе я поеду..."
1,- я дам вам массаж,- я [f-red]сделаю[/f-red] вам массаж,Russian,- я сделаю вам массаж
2,"Поэтому, я волновалась до того, как моя голова...","Поэтому, я волновалась до того, [f-red]что[/f-...",Russian,"Поэтому, я волновалась до того, что моя голова..."
3,"В результате, у меня есть круглый, крупный живот.",[sline][f-blue][/f-blue][/sline][f-blue]Теперь...,Russian,Теперь у меня большой круглый живот.
4,Я не понимал все...,Я [f-bold]всё [/f-bold]не понимал [sline]все[/...,Russian,Я всё не понимал ...
